In [2]:
import pymongo as pm
import requests
import pandas as pd 
pd.options.plotting.backend = "plotly"
# yahoo_fin documentation: https://theautomatic.net/yahoo_fin-documentation/
from yahoo_fin import stock_info as si 
from pandas_datareader import DataReader
import numpy as np

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import plotly.graph_objects as go

/Users/yegor/anaconda3/envs/springboard/lib/python3.8/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
from datetime import date
from newspaper import Article

In [4]:
import nltk
# need to do once for vader to work
nltk.download('vader_lexicon')
# need to do once for newspaper to work
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/yegor/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/yegor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import plotly.io as pio
# check available plotly renderers
pio.renderers

# set pandas plotting backend to plotly
pd.options.plotting.backend = "plotly"

# options for plotly to work in the notebook
pio.renderers.default = 'notebook'

In [6]:
def plot_ticker_data(ticker_data, xaxis_rangeslider_visible=True):
    """
    Plot candelstick
    """
    
    fig = go.Figure(data=[go.Candlestick(
        x=ticker_data.index,
        open=ticker_data.open,
        high=ticker_data.high,
        low=ticker_data.low,
        close=ticker_data.close,
#         increasing_line_color= 'blue', 
#         decreasing_line_color= 'red'
    )])

    # uncomment below to remove rangeslider
    fig.update_layout(xaxis_rangeslider_visible=xaxis_rangeslider_visible)

#     fig.show()
    return fig

In [ ]:
ticker = 'MSFT'
start_date = '01/01/2015'
end_date = '05/31/2020'
ticker_data = si.get_data(ticker, start_date=start_date, end_date=end_date)
plot_ticker_data(ticker_data)

In [ ]:
# Start MongoDB
!brew services start mongodb-community@4.2

In [ ]:
# Stop MongoDB
!brew services stop mongodb-community@4.2

In [7]:
def get_recommendation(ticker):
    """
    Obtaines yahoo recommendations for a 'ticker'
    """
    lhs_url = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
    rhs_url = '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
              'modules=upgradeDowngradeHistory,recommendationTrend,' \
              'financialData,earningsHistory,earningsTrend,industryTrend&' \
              'corsDomain=finance.yahoo.com'
              
    url =  lhs_url + ticker + rhs_url
    r = requests.get(url)
    if not r.ok:
        recommendation = 6
    try:
        result = r.json()['quoteSummary']['result'][0]
        recommendation =result['financialData']['recommendationMean']['fmt']
    except:
        recommendation = 6
    
    return recommendation

In [8]:
def get_and_store_recommendations(ticker, dt=None):
    """
    Retrieves yahoo recommendations for a 'ticker' and stores to MongoDB avoiding duplicates
    """
    client = pm.MongoClient('mongodb://localhost:27017')
    collection = client['news']['recommendations']
    
    day = date.today().strftime('%Y-%m-%d') if dt is None else dt

    doc = {
        'recommendations': {
            'date' : day,
            'recommendation' : get_recommendation(ticker)
        }
    }
    
    collection.update_one(
        {'ticker' : ticker},
        {'$addToSet': doc},
        upsert = True
    )
    print(f"Saved {ticker}: {doc}")
    return doc

In [ ]:
def get_and_store_news(ticker):
    """
    Obtaines news for the 'ticker' and save to MongoDB
    """
    logger.info(f'Working on {ticker}')
    client = pm.MongoClient('mongodb://localhost:27017')
    collection = client['news']['recommendations']
    finwiz_url = 'https://finviz.com/quote.ashx?t='
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    response = urlopen(req)    
    html = BeautifulSoup(response)
    news_table = html.find(id='news-table')

    for x in news_table.findAll('tr'):

        title = x.a.get_text() 
        link = x.a['href']
        
        date_scrape = x.td.text.strip().split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        dt = date + ' ' + time
        try:
            article = Article(link)
            article.download()
            article.parse()
            article.nlp()
        except:
            continue
        
        doc = {
            'news' : {
                'datetime' : dt,
                'url'      : link,
                'title'    : title,
                'text'     : article.text,
                'keywords' : article.keywords,
                'summary'  : article.summary
            }
        }

        collection.update_one(
            {'ticker' : ticker},
            {'$addToSet': doc},
            upsert = True
        )
        
        print(f"Saved {ticker}: {dt}/ {doc['news']['title']}")
        


if __name__ == '__main__': 
    logging.basicConfig(level=logging.DEBUG)
    logger.info('Launching multi')
    tickers = si.tickers_sp500()
    
    p = Pool(10)
    result = p.map_async(get_and_store_news, tickers)
    result.get()


In [12]:
# get the SP500 list
tickers = si.tickers_sp500()

In [18]:
import multiprocessing

print("Number of cpu : ", multiprocessing.cpu_count())

Number of cpu :  8


In [14]:
# retrieve and store current recommendations
for ticker in tickers: 
#     print(get_and_store_recommendations(ticker))
    get_and_store_news(ticker, collection)

NameError: name 'collection' is not defined

In [ ]:
get_and_store_news('AAPL')

In [ ]:
df = pd.DataFrame(list(zip(tickers, recommendations)), 
               columns =['company', 'recommendation']) 
df.head()

In [ ]:
df['recommendation'] = pd.to_numeric(df['recommendation'])
df = df[df.recommendation != 6]
df.sort_values(by=['recommendation'], ascending = True)

For the purpose of this analysis, the only values that are of importance are 1–1.5, 3, and 4.5–5. These are the scores, that signify the highest chance for an event to take place, and thus are the best indicators..
only specific values are of importance (the highest indicators). As a result, I will be creating three novel dataframes, named “hold_df”, “buy_df”, and “sell_df” that will then be contacted into “new_df”.

In [ ]:
hold_df = df[df.recommendation == 3]
buy_df = df[df.recommendation <= 1.5]
sell_df = df[df.recommendation >= 4.5]

df_list = [hold_df, buy_df, sell_df]
new_df = pd.concat(df_list)
new_df.reset_index(level=0, inplace=True)
new_df

It is now time to fetch the News and Twitter feed. Once both feeds are successfully fetched, sentiment analysis for each stock will be individually conducted for each platform and then the two results will be added and divided by two.
So the final sentiment score will be calculated as follows:
Final Score = (Twitter Sentiment Score + News Feed Sentiment Score) / 2

In [ ]:
tickers =[] 
  
for index, rows in new_df.iterrows(): 
    tickers.append(rows.company) 

print(tickers)

finviz is going to be used to parse the news data into a Pandas dataframe.

In [ ]:
finwiz_url = 'https://finviz.com/quote.ashx?t='

news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    response = urlopen(req)    
    html = BeautifulSoup(response)
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table
    
parsed_news = []

for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
    
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])
        

In [ ]:
columns = ['ticker', 'date', 'time', 'headline']

parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)

parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date

parsed_and_scored_news.head()

The problem with the data is that in their current form, they can not be used by any model. Thus, I will be grouping the headlines for each company in one string, according to the company they are referring to.

In [ ]:
parsed_and_scored_news = parsed_and_scored_news.groupby(['ticker'], as_index = False)\
    .agg({'headline': ''.join}, Inplace=True)
parsed_and_scored_news

In [ ]:
vader = SentimentIntensityAnalyzer()

scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()

scores_df = pd.DataFrame(scores)

parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')

In [ ]:
parsed_and_scored_news

In [ ]:
# download language model
!python -m spacy download en_core_web_sm

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

# for token in doc:
#     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#             token.shape_, token.is_alpha, token.is_stop)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
import pandas as pd

In [ ]:
!ls stocks_latest

In [ ]:
df = pd.read_csv('dataset_summary.csv')
df.head()

In [ ]:
df = pd.read_csv('stocks_latest/stock_prices_latest.csv', parse_dates=['date'])
df.head()

In [ ]:
df.info()

In [ ]:
stock = ['A']
A = df[df.symbol.isin(stock)]

In [ ]:
A.set_index('date', inplace=True)
A.sort_index(inplace=True)

In [ ]:
A.head()

In [ ]:
A.truncate(before='2015-01-01')['close'].plot(figsize=(16,12))

In [ ]:
by_symbol = df.groupby(['date','symbol'])

In [ ]:
by_symbol